In [ ]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import pickle

# Load the trained model
with open('yoga_pose_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp_solutions.drawing_utils

st.title("Yoga Pose Detection")
st.write("This app uses your webcam to detect poses using a trained ML model.")

cap = cv2.VideoCapture(0)
frame_placeholder = st.empty()
stop_button_pressed = st.button("Stop")

while cap.isOpened() and not stop_button_pressed:
    ret, frame = cap.read()
    
    # Check if the frame is not empty
    if not ret or frame is None:
        continue 

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Process the image and get the pose landmarks.
    results = pose.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # --- MODEL PREDICTION LOGIC ---
    if results.pose_landmarks:
        # Draw the pose annotation on the image.
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )
        
        try:
            # Extract landmarks for the model
            landmarks = results.pose_landmarks.landmark
            row = []
            for lm in landmarks:
                row.extend([lm.x, lm.y, lm.z, lm.visibility])
            
            # Make a prediction
            X = np.array(row).reshape(1, -1)
            pose_name = model.predict(X)[0]
            
            # Display the prediction
            cv2.putText(image, pose_name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        except Exception as e:
            cv2.putText(image, "Error", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
    frame_placeholder.image(image, channels="BGR", use_container_width=True)

cap.release()
cv2.destroyAllWindows()